# Predict parent neuron
Using Jasper's idea to predict parent neurons for nuclei, Thank you so much, Jasper.

In [1]:
# libraries 1
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui.statebuilder import *
import json
from tqdm import tqdm
import argparse

from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import csv

from tifffile.tifffile import imwrite

sys.path.append(os.path.abspath("../segmentation"))
import authentication_utils as auth
import rootID_lookup as IDlook

In [2]:
np.random.seed(123)

In [3]:
# read csv file
df = pd.read_csv('../Output/body_info.csv', header=0)
len(df)

17134

In [8]:
# Create Neuroglancer links
# https://github.com/bjm5164/rotation_projects/blob/main/statebuilder_examples.ipynb
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc = ImageLayerConfig(name = 'nuclei_Jul2021',
                                    source = auth.get_cv_path('nuclei_map_Jul2021')['url'])      
nuc_seg = ImageLayerConfig(name = 'nuclei_Jul2021_seg',
                                    source = auth.get_cv_path('nuclei_seg_Jul2021')['url'])      
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

LINK=[]
k=100
minidfs = [df.loc[i:i+k-1, :] for i in range(0, len(df), k)]
for dftemp in minidfs:
    seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                        active = True,
                                        source = auth.get_cv_path('FANC_production_segmentation')['url'],
                                        selected_ids_column='nuc_segID')                                 
    sb = StateBuilder(layers=[imgTokyo, seg, img, nuc, nuc_seg],resolution=[4.3,4.3,45]) # view_kws=view_options
    output = sb.render_state(dftemp) #return_as='html'
    LINK.append(output)

In [9]:
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/nuc_links.csv', index=False, header=False)

In [9]:
# T1MN_somas test
MNsomas = pd.read_csv('../Output/T1MN_somas.csv', header=0)
MNsomas['pt_root_id'].values

exist = []
no_exist = []
for i in MNsomas['pt_root_id'].values:
    if i in df['body_segID'].values:
        exist.append(i)
    else:
        no_exist.append(i)

print(no_exist)

[648518346486520985, 648518346470147995, 648518346472220107, 648518346489586393, 648518346488111357, 648518346473993325, 648518346484103178, 648518346499209563, 648518346500966285, 648518346488109309, 648518346489861073, 648518346497274037]


In [13]:
check = 648518346486520985
print (df.loc[df['body_segID'] == check])
print (MNsomas.loc[MNsomas['pt_root_id'] == check])

Empty DataFrame
Columns: [blockID, x, y, z, nuc_segID, nucID, size_x_mip4, size_y_mip4, size_z_mip4, vol, body_segID]
Index: []
    Unnamed: 0         name  cell_type           pt_position  \
20          20  MN_A402_T1L        NaN  [19270, 88811, 2289]   

            pt_root_id  soma_x_nm  soma_y_nm  soma_z_nm  found  table_name  
20  648518346486520985    82861.0   381887.3   103005.0    NaN  T1MN_somas  


In [14]:
nuc = 648518346481315476
df[df['nuc_segID']==nuc]

,blockID,x,y,z,nuc_segID,nucID,size_x_mip4,size_y_mip4,size_z_mip4,vol,body_segID
16811,6367,18928,87968,2288,648518346481315476,72340512660524829,120,90,133,1436400,648518346500131388


In [11]:
df[df['nuc_segID']==648518346491474694]

,blockID,x,y,z,nuc_segID,nucID,size_x_mip4,size_y_mip4,size_z_mip4,vol
23,4144,31480,75464,1487,648518346491474694,72551412533297182,31,41,52,66092


In [12]:
df[df['nuc_segID']==648518346447572945]

,blockID,x,y,z,nuc_segID,nucID,size_x_mip4,size_y_mip4,size_z_mip4,vol
9,4144,31512,74784,1476,648518346447572945,72551412533297246,32,26,64,53248


In [ ]:
count = pd.read_csv('../Output/ncount_merged.csv', header=None)
count2 = count.T.dropna()
# len(count2) 
# 15094
count3 = count2.sort_values(by=0)
count4 = count3.astype('int')
count4.columns = ['index', 'count']

In [ ]:
sv_size_thres=4000
chosen = count4.loc[count4['count'] <= sv_size_thres]
chosen['index'].values

In [ ]:
test = df.loc[chosen['index'].values]
test[0:999]
rantest = test.iloc[np.random.choice(len(test), round(len(test)/5), replace=False)]

In [ ]:
# nuclei itself
# Create Neuroglancer links
imgTokyo = ImageLayerConfig(name = 'FANC_EM_Tokyo',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANC_EM',
                                    source = auth.get_cv_path('Image')['url'])
nuc = ImageLayerConfig(name = 'nuclei(old)',
                                    source = auth.get_cv_path('nuclei_map')['url'])                                    
# volume = SegmentationLayerConfig(name = 'volume_outlines',
#                                    source = auth.get_cv_path('Image')['url'])   

seg = SegmentationLayerConfig(name = 'FANC_production_segmentation',
                                        active = True,
                                        source = auth.get_cv_path('FANC_production_segmentation')['url'],
                                        selected_ids_column='segIDs')                                 
sb = StateBuilder(layers=[imgTokyo, seg, nuc, img],resolution=[4.3,4.3,45]) # view_kws=view_options
output = sb.render_state(rantest, return_as='html')
output

In [ ]:
sv_size_thres=4000
notchosen = count4.loc[count4['count'] > sv_size_thres+2000]
test2 = df.loc[notchosen['index'].values]
test2

In [ ]:
rantest2 = test2.iloc[np.random.choice(len(test2), round(len(test2)/5), replace=False)]
output = sb.render_state(rantest2, return_as='html')
output

In [ ]:
# https://api.zetta.ai/json/post

In [ ]:
# skeleton ID to segID

In [ ]:
skel = [254234,254426,254765,254781,254687,254570,254411,254361,254495,254750,254268,254199,255121,255197,255062,254941,254524,254510,254637,254656,255033,255092,255154,255243,254814,254392,254797,254377,254620,255077,254846,254911,254971,255137,254474,254602,255228,254862,254457,254926,255182,254329,254672,254587,255167,255256,254958,254986,255048,255107,254894,255018,254718,254539,254703,254831,254878,254733,254252,254442,254555,254284,254300,255269,254315,255001,255212,254345,254217,396992,397012,397053,396860,396948,397033,397073,396968,396882,397094,396923,396904,441325,441232,441128,441213,395088,395244,395216,395129,395046,394970,395150,395364,394821,394929,395294,395804,395739,395831,395068,395382,394991,394844,395849,394867,395871,394797,394888,395655,396007,395194,395949,395761,395695,395171,395781,395009,394910,395026,395714,395271,395410,395111,394951,395969,395912,395893,395451,396224,396069,396187,396205,396150,396108,396028,396128,396089,396050,396168]

In [ ]:
# sys.path.append(os.path.abspath("../proofreading"))
# import proofreading_utils
sys.path.append(os.path.abspath("../transforms"))
import realignment
sys.path.append(os.path.abspath("../skeletonization"))
import catmaid_utilities

In [ ]:
def skel2seg(neuron,
             target_volume,
             transform=True):
    nodes = neuron.nodes[['x','y','z']].values/ np.array([4.3,4.3,45])
    
    if transform is True:
        points = realignment.fanc3_to_4(nodes)
    else:
        points = nodes
    
    return IDlook.segIDs_from_pts_service(points),points

In [ ]:
def skel2scene(skid, project=13):
    
    CI = catmaid_utilities.catmaid_login('FANC',project)
    try:
        n = pymaid.get_neurons(skid)
    except:
        return('No matching skeleton ID in project {}'.format(project))
    
    n.downsample(inplace=True)

    target_volume = CloudVolume(auth.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False)
    seg_ids,points = skel2seg(n, target_volume, transform=True)
    return seg_ids[0]